# Question 2
Consider the Mammography dataset available on the resources tab. There are two classes: class 1 indicates calcification (cancer) and class 0 indicates no calcification (no cancer). Thus, the class 1 is the positive class and class 0 is the negative class. You are required to use and compare a neural network classifier (MLPClassifier in scikit-learn, for example) and a decision tree classifier (DecisionTreeClassifier in scikit-learn, for example). You will use 10-fold cross-validation (StratifiedKFold in scikit-learn; also look at cross_val_score) to compare the two classifiers. Please identify classifier is statistically significantly better at 95% confidence when using Error as a metric. Please identify which classifier is statistically significantly better at 95% confidence when using AUC or F-measure as a metric.  Please also discuss if there are any differences in classifier performance when using AUC / F-measure or Error as the evaluation metric.  (30 points)

*Extra Credit: Consider optimizing the decision tree pruning criterion or MLP learning rate / number of units and see if the performance can be improved. (5 points)*